In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import os
import cv2
from PIL import Image
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten

## **Data Read**

In [ ]:
!pip install opendatasets
!pip install pandas

In [ ]:
import opendatasets as od
import pandas as pd

od.download("https://www.kaggle.com/datasets/norannabil/drowness-detection")

In [ ]:
dataset=[]
label=[]
SIZE=224

sleep= os.listdir('/content/drowness-detection/data_cropped/Sleep/')
for i, image_name in enumerate(sleep):

        image = cv2.imread('/content/drowness-detection/data_cropped/Sleep/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)


In [ ]:
len(label)

In [ ]:
active= os.listdir("/content/drowness-detection/data_cropped/Active/")
for i, image_name in enumerate(active):
        image = cv2.imread('/content/drowness-detection/data_cropped/Active/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)



In [ ]:
len(label)

In [ ]:
yawn= os.listdir("/content/drowness-detection/data_cropped/Yawn/")
for i, image_name in enumerate(yawn):
        image = cv2.imread('/content/drowness-detection/data_cropped/Yawn/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(2)

In [ ]:
len(label)

In [ ]:
dataset = np.array(dataset)
label = np.array(label)

In [ ]:
dataset.shape,label.shape

# **Train Valid Test split**

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.25, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.25, random_state = 42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape,X_val.shape, y_val.shape,

# **Model and Training**

In [ ]:
from keras.applications import VGG16
VGG16_model = Sequential()

prdeletrained_mo= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=3,
                   weights='imagenet')
for layer in prdeletrained_mo.layers:
        layer.trainable=False

VGG16_model.add(prdeletrained_mo)
VGG16_model.add(Flatten())
VGG16_model.add(Dense(512, activation='relu'))
VGG16_model.add(Dense(512, activation='relu'))
VGG16_model.add(Dropout(0.4))
VGG16_model.add(Dense(3, activation='softmax'))

In [ ]:
VGG16_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 15,241,539
Trainable params: 526,851
Non-t

In [ ]:
VGG16_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.models import Model
history = VGG16_model.fit(X_train,
                         y_train,
                         batch_size = 32,
                         verbose = 2,
                         epochs = 30,
                         validation_data=(X_val,y_val),
                         shuffle = False
                     )

In [ ]:
from sklearn import metrics

In [ ]:
VGG16_model.save('/content/drive/MyDrive/ai-iot-project/ai/weights/Vgg16_cropped_AUG_final.h5') ### Replace with path to save the model weights

In [ ]:
y_pred =VGG16_model.predict(X_test)

In [ ]:
y_pred =[np.argmax(pr) for pr in y_pred]

In [ ]:
print("Test accuracy : ")
print(metrics.accuracy_score(y_test, y_pred))